In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import time

In [2]:
# Le début des relevés est 2013. 

def get_all_data():
    base_url = "https://hubeau.eaufrance.fr/api/v1/vente_achat_phyto/achats/produits"
    params = {
        "type_territoire": "Département",
        "annee_min": 2012,
        "annee_max": 2013,
        "size": 1000,
        "page": 1
    }

    all_data = []

    print(" Accès à l'API en cours... Veuillez patienter.")
    
    start_time = time.time()
    
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data_page = response.json().get("data", [])
        
        if not data_page:
            break
        
        all_data.extend(data_page)
        params["page"] += 1

        
    end_time = time.time()
    duration = end_time - start_time

    print(f"Nombre de fichiers récupérés :{len(all_data)}")
    return pd.DataFrame(all_data)


In [3]:
df = get_all_data()

 Accès à l'API en cours... Veuillez patienter.
Nombre de fichiers récupérés :64459


In [6]:
df.head()

,achat_etranger,annee,amm,code_territoire,eaj,libelle_territoire,type_territoire,uri_territoire,quantite,unite
0,Non,2013,2020346,79,Non,DEUX-SEVRES,Département,http://id.eaufrance.fr/dep/79,109.0,l
1,Non,2013,2010502,84,Non,VAUCLUSE,Département,http://id.eaufrance.fr/dep/84,13.0,l
2,Non,2013,2030011,13,Non,BOUCHES-DU-RHONE,Département,http://id.eaufrance.fr/dep/13,1.0,l
3,Non,2013,2010612,32,Oui,GERS,Département,http://id.eaufrance.fr/dep/32,9660.0,l
4,Non,2013,2010633,47,Non,LOT-ET-GARONNE,Département,http://id.eaufrance.fr/dep/47,110.0,l


In [7]:
#conservation des colonnes utiles pour augmenter la lisibilité. 
df_propre = df[["amm","annee","code_territoire","eaj","libelle_territoire","quantite", "unite"]]

In [8]:
df_propre.head()

,amm,annee,code_territoire,eaj,libelle_territoire,quantite,unite
0,2020346,2013,79,Non,DEUX-SEVRES,109.0,l
1,2010502,2013,84,Non,VAUCLUSE,13.0,l
2,2030011,2013,13,Non,BOUCHES-DU-RHONE,1.0,l
3,2010612,2013,32,Oui,GERS,9660.0,l
4,2010633,2013,47,Non,LOT-ET-GARONNE,110.0,l


In [13]:
# Je vais maintenant supprimer les départements ayant le code territoire 0, 
# représentant une erreur dans la saisie. 
len(df_propre)

64459

In [15]:
(df_propre["code_territoire"] == "0").sum()

np.int64(0)

In [16]:
df_propre = df_propre[df_propre["code_territoire"] != "0"]

In [17]:
len(df_propre)

64459

In [18]:
# Je vais maintenant convertir les codes en int et changer les noms pour
# les rendre plus compréhensibles
df_propre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64459 entries, 0 to 64458
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   amm                 64459 non-null  object 
 1   annee               64459 non-null  int64  
 2   code_territoire     64459 non-null  object 
 3   eaj                 64459 non-null  object 
 4   libelle_territoire  64459 non-null  object 
 5   quantite            64459 non-null  float64
 6   unite               64459 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 3.4+ MB


In [20]:
df_propre = df_propre.rename(columns={
    "code_territoire": "num_département",
    "libelle_territoire": "département",
    "eaj" : "autorise_jardin",
    "quantite": "quantite_en_kg",
})

df_propre.head()

,amm,annee,num_département,autorise_jardin,département,quantite_en_kg,unite
0,2020346,2013,79,Non,DEUX-SEVRES,109.0,l
1,2010502,2013,84,Non,VAUCLUSE,13.0,l
2,2030011,2013,13,Non,BOUCHES-DU-RHONE,1.0,l
3,2010612,2013,32,Oui,GERS,9660.0,l
4,2010633,2013,47,Non,LOT-ET-GARONNE,110.0,l


In [22]:
colonnes_entiers = ["amm", "num_département"] 
for col in colonnes_entiers:
    df_propre[col] = pd.to_numeric(df_propre[col], errors="coerce").astype("Int64")

In [24]:
df_propre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64459 entries, 0 to 64458
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   amm              64459 non-null  Int64  
 1   annee            64459 non-null  int64  
 2   num_département  64304 non-null  Int64  
 3   autorise_jardin  64459 non-null  object 
 4   département      64459 non-null  object 
 5   quantite_en_kg   64459 non-null  float64
 6   unite            64459 non-null  object 
dtypes: Int64(2), float64(1), int64(1), object(3)
memory usage: 3.6+ MB


Une fois les réglages effectués, on passe au script python qui va enregistrer en base de données. Lancer le script Insertion_data.py